**파일이 매우 큽니다. Colab pro를 이용하여 고용량 RAM을 사용하시는 것이 속 편합니다. <br>
Colab pro 구독을 했다면, 노트 설정에서 대용량 RAM으로 바꾸시면 됩니다.**



---------------------------



**구글 드라이브 연동입니다. <br>
/content/drive로 mount를 했습니다. 그러기에 마지막 input_file, output_file의 절대 경로를 자신에게 잘 맞춰 입력해주십시오.**

In [ ]:
from google.colab import drive
drive.mount('gdrive')
#구글드라이브연동

---------------------------

In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
from datetime import datetime

In [ ]:
dfs = []
with ZipFile('/content/gdrive/MyDrive/lecture-2023-bigdata/인천광역시_운행속도_202001.zip', 'r') as zipObj:
  listOfFileNames = zipObj.namelist()
  for fileName in listOfFileNames:
      if fileName.endswith('csv'):
          print(fileName)
          with zipObj.open(fileName) as f:
            for chunk in pd.read_csv(f, chunksize=10 ** 6, encoding='cp949'):
                dfs.append(chunk)
                del chunk
                gc.collect()
            f.close()
  zipObj.close()

taxi = pd.concat(dfs)
taxi["시"].unique()

In [ ]:
un = taxi["시"].unique()
un

**csv 파일 전처리 함수 정의입니다. 필요 없는 column은 제거하였으며, 필요한 column을 생성하여 insert 하였습니다.**

In [ ]:
import io
from zipfile import ZipFile
import gc




chunksize = 10 ** 6 # 한번에 읽을 데이터 크기

# usecols = ['시간', '동', '경도', '운행속도'] usecols=usecols,

def preprocess_csv(input_file, output_file):
    dfs = []
    with ZipFile(input_file, 'r') as zipObj:
      listOfFileNames = zipObj.namelist()
      for fileName in listOfFileNames:
          if fileName.endswith('csv'):
              print(fileName)
              with zipObj.open(fileName) as f:
                for chunk in pd.read_csv(f, chunksize=chunksize, encoding='cp949'):
                    dfs.append(chunk)
                    del chunk
                    gc.collect()
                f.close()
      zipObj.close()

    taxi = pd.concat(dfs)
    taxi["시"].unique()
    day = pd.to_datetime(taxi['시간'], format='%Y-%m-%d %H:%M:%S').dt.date
    time = pd.to_datetime(taxi['시간'], format='%Y-%m-%d %H:%M:%S').dt.hour
    week = pd.to_datetime(day, format='%Y-%m-%d').dt.dayofweek
    month = pd.to_datetime(taxi['시간'], format='%Y-%m-%d %H:%M:%S').dt.month

    taxi = taxi.drop('시간', axis=1)
    taxi.insert(1, '요일', week)
    taxi.insert(2, '시간', time)
    taxi.insert(3, '월', month)

    taxi = taxi.drop(['센서명','시', '군구'], axis=1)

    def modify_time_category(time):
        if 1 <= time <= 5:
            return 0
        elif 6 <= time <= 9:
            return 1
        elif 10 <= time <= 15:
            return 2
        elif 16 <= time <= 19:
            return 3
        else:
            return 4

    taxi['시간대'] = taxi['시간'].apply(modify_time_category)
    taxi = taxi.drop('시간', axis=1)

    idx = taxi[taxi['운행속도'] == 0].index
    taxi.drop(idx, axis=0, inplace=True)

    taxi['위도'] = taxi['위도'].astype('float').round(5)
    taxi['경도'] = taxi['경도'].astype('float').round(5)

    taxi = taxi.groupby(['요일', '월', '시간대', '동', '위도', '경도'], as_index=False)['운행속도'].mean()
    taxi['운행속도'] = taxi['운행속도'].astype('int')

    taxi.to_csv(output_file, index=False)

**직접 input_file과 output_file의 경로를 입력하여 dataframe으로 만들어 preprocessing을 실행하는 단계입니다.**

In [ ]:
# 입력 및 출력 파일 이름을 지정하여 preprocess_csv 함수를 호출합니다.
input_file = '/content/gdrive/MyDrive/lecture-2023-bigdata/인천광역시_운행속도_202001.zip'
output_file = '/content/gdrive/MyDrive/lecture-2023-bigdata/resultC_202001.csv'
preprocess_csv(input_file, output_file)

---------

**여러개는 어떻게 할까?**

In [ ]:
def preprocess_csv_many(input_files, output_files):
     def modify_time_category(time):
        if 1 <= time <= 5:
            return 0
        elif 6 <= time <= 9:
            return 1
        elif 10 <= time <= 15:
            return 2
        elif 16 <= time <= 19:
            return 3
        else:
            return 4

     for i in range(len(input_files)):
        input_file = input_files[i]
        output_file = output_files[i]

        taxi = pd.read_csv(input_file, encoding='CP949', header=0)

        day = pd.to_datetime(taxi['시간'], format='%Y-%m-%d %H:%M:%S').dt.date
        time = pd.to_datetime(taxi['시간'], format='%Y-%m-%d %H:%M:%S').dt.hour
        week = pd.to_datetime(day, format='%Y-%m-%d').dt.dayofweek
        month = pd.to_datetime(taxi['시간'], format='%Y-%m-%d %H:%M:%S').dt.month

        taxi = taxi.drop('시간', axis=1)
        taxi.insert(1, '요일', week)
        taxi.insert(2, '시간', time)
        taxi.insert(3, '월', month)

        taxi = taxi.drop(['센서명', '위도', '경도', '지오해시', '시', '군구'], axis=1)



        taxi['시간대'] = taxi['시간'].apply(modify_time_category)
        taxi = taxi.drop('시간', axis=1)

        idx = taxi[taxi['운행속도'] == 0].index
        taxi.drop(idx, axis=0, inplace=True)

        taxi = taxi.groupby(['요일', '월', '시간대', '동'], as_index=False)['운행속도'].mean()
        taxi['운행속도'] = taxi['운행속도'].astype('int')

        taxi.to_csv(output_file, index=False)

In [ ]:
input_files = [
    '/content/drive/MyDrive/smubigdata2023/ich_202108.csv',
    '/content/drive/MyDrive/smubigdata2023/ich_202109.csv',
    '/content/drive/MyDrive/smubigdata2023/ich_202110.csv'
]
output_files = [
    '/content/drive/MyDrive/smubigdata2023/result_2108.csv',
    '/content/drive/MyDrive/smubigdata2023/result_2109.csv',
    '/content/drive/MyDrive/smubigdata2023/result_2110.csv'
]
preprocess_csv_many(input_files, output_files)

---------

**병합 부분입니다. <br> 병합은 안터지는 한에서 그냥 최대한 합치면 몇번 안돌리고 후딱 끝낼겁니다. <br>전처리를 위에서 다해놔서 합쳐도 제거할거나 중복이 없거든요. "month" column 때문에 <br> 그래서 output_file에서 파일명도 직접 바꾸면 됩니다. 그냥 지금은 merge_21_시작_끝.csv로 임시로 써놨습니다**

In [ ]:
def merge_csv(input_files, output_file):
    # 빈 DataFrame 생성
    merged_df = pd.DataFrame()

    for input_file in input_files:
        # CSV 파일 읽어오기
        df = pd.read_csv(input_file)

        # 읽어온 DataFrame을 빈 DataFrame에 추가
        merged_df = pd.concat([merged_df, df], ignore_index=True)

    # 합쳐진 DataFrame을 CSV 파일로 저장
    merged_df.to_csv(output_file, index=False)

In [ ]:
# 입력 파일 리스트와 출력 파일 이름을 지정합니다.
input_files = [
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202001.csv',
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202002.csv',
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202003.csv',
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202004.csv',
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202005.csv',
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202006.csv',
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202007.csv',
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202008.csv',
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202009.csv',
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202010.csv',
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202011.csv',
    '/content/gdrive/MyDrive/lecture-2023-bigdata/result_202012.csv',
]
output_file = '/content/gdrive/MyDrive/lecture-2023-bigdata/merge_20.csv'

# merge_csv 함수 호출
merge_csv(input_files, output_file)

---------

**데이터 확인**

In [ ]:
merge=pd.read_csv('/content/gdrive/MyDrive/lecture-2023-bigdata/merge_20_01_02.csv')

In [ ]:
merge